In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from utils import regression_model
from sklearn.preprocessing import StandardScaler
import pickle

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

compilation of mae_val score of all NN models in step 2

In [2]:
score4 = pd.read_csv('./submission/predictions/NN_emb32_score_try4.csv', index_col=0)
score5 = pd.read_csv('./submission/predictions/NN_emb32_score_try5.csv', index_col=0)
score6 = pd.read_csv('./submission/predictions/NN_emb32_score_try6.csv', index_col=0)
score7 = pd.read_csv('./submission/predictions/NN_emb32_score_try7.csv', index_col=0)
score8 = pd.read_csv('./submission/predictions/NN_emb32_score_try8.csv', index_col=0)

score = pd.concat([score4, score5, score6, score7, score8])
score.reset_index(inplace=True)

precomputed gene embedding

In [3]:
emb_size = 32
g_embed = np.load(f'submission/embedding/gene_embedding_{emb_size}.npy')
g_name = np.load(f'submission/embedding/gene_names_{emb_size}.npy', allow_pickle= True)

transform the gene embedding by StandardScaler

In [4]:
with open('./submission/scaler.pkl', 'rb') as f:
    X_scaler = pickle.load(f)

#transform test set
X_test_transformed = X_scaler.transform(g_embed)

compute predictions of all 15077 genes of all best 2 models for each k-fold dataset

In [ ]:
best_n = 2
min_mae_val=0.10

predictions = {}

for k in range(10):
    predictions[k] = []
    sc = score[score.k==k].sort_values('mae_val').head(best_n)
    sc = sc[sc.mae_val < min_mae_val]
    sc['l1'] = [str(i) if i != 0.0 else '0' for i in sc.l1]
    sc['l2'] = [str(i) if i != 0.0 else '0' for i in sc.l2]
    
    for i in range(len(sc)):
        emb_size = 32
        dropout_rate = sc.dropout_rate.tolist()[i]
        l1 = sc.l1.tolist()[i]
        l2 = sc.l2.tolist()[i]
        learning_rate = sc.learning_rate.tolist()[i]
        mae_weight = sc.mae_weight.tolist()[i]
        batch_size = sc.batch_size.tolist()[i]
        k = sc.k.tolist()[i]
        n_try = sc.n_try.tolist()[i]
        model_name = f"model_{emb_size}_{dropout_rate}_{l1}_{l2}_{learning_rate}_{mae_weight}_{batch_size}_{k}_{n_try}"
        
        #load corresponding model
        model_path = f"./submission/checkpoints/NN/{model_name}.h5"
        model = regression_model(
            input_size=32,
            output_size=5,
            size_dense=8,
            dropout_rate=dropout_rate,
            l1=float(l1),
            l2=float(l2),
            learning_rate=learning_rate,
            mae_weight=2
        )
        model.load_weights(model_path)
        Y_test_pred = model.predict(X_test_transformed, verbose=0)
        predictions[k].append(Y_test_pred)
    
    predictions[k] = np.array(predictions[k])

In [6]:
prop = pd.read_csv("training_cell_state_proportions.csv", index_col=0)
g_train = prop.index.tolist()
g_train = [g for g in g_train if g in g_name]
print(len(g_train))

64


In [7]:
def make_final_prediction(g, min_distance = 0.06):
    print(f'prediction gene {g}:') 
    ## initiate the list to collect prediction of this gene
    p = np.array([0])
    
    ## collect prediction from 20 models of 10 k-fold training
    for k in range(10):
        if k==0:
            p = predictions[k][:,list(g_name).index(g),:]
        else:
            p = np.concatenate([p, predictions[k][:,list(g_name).index(g),:]], axis=0)
            
    ## calculate MAE for each pair within 20 predictions, 
    ## and select the predictions with highest number of similar predictions (mae<0.06)
    
    def count_similar_vectors(p, min_distance):
        distance = {}

        # calculate l1-distance/5 for each pair
        for i in range(len(p)):
            distance[i] = []
            for j in range(len(p)):
                distance[i].append(mean_absolute_error(p[i], p[j]))

        # count similar vectors
        n_similar_node = [len([n for n in distance[i] if n<min_distance])-1 for i in range(len(p))]
        print(f"--max number of similar vector for min_distance {min_distance}:", max(n_similar_node))
        
        most_neighbors_nodes = [i for i in range(len(p)) if n_similar_node[i] == max(n_similar_node)]
        most_neighbors_dists = p[most_neighbors_nodes]
        return most_neighbors_dists
    
    def l1_distance_between_vectors(most_neighbors_dists):
        mae_of_similar_nodes = []
        for i in range(len(most_neighbors_dists)):
            for j in range(len(most_neighbors_dists)):
                mae_of_similar_nodes.append(mean_absolute_error(most_neighbors_dists[i], most_neighbors_dists[j]))
        return mae_of_similar_nodes
    
    # make sure all vectors are similar
    while True:
        most_neighbors_dists = count_similar_vectors(p, min_distance)
        if len(most_neighbors_dists) <= 1:
            break
        mae_of_similar_nodes = l1_distance_between_vectors(most_neighbors_dists)
        if max(mae_of_similar_nodes) <= min_distance:
            break
        min_distance -= 0.005
        print(f"---min_distance: {min_distance}")
    
    print("-", np.array(most_neighbors_dists))
    g_pred = np.mean(np.array(most_neighbors_dists), axis = 0)
    print("-mean: ", g_pred, "\n")
    
    return g_pred

In [8]:
prediction_all_genes = []
min_distance = 0.06

for g in g_name:
    y_pred = make_final_prediction(g, min_distance)
    prediction_all_genes.append(y_pred)

prediction gene Mrpl15:
--max number of similar vector for min_distance 0.06: 10
- [[0.157 0.233 0.250 0.357 0.003]]
-mean:  [0.157 0.233 0.250 0.357 0.003] 

prediction gene Lypla1:
--max number of similar vector for min_distance 0.06: 6
- [[0.456 0.151 0.196 0.194 0.003]
 [0.557 0.131 0.147 0.147 0.019]
 [0.520 0.192 0.142 0.145 0.001]]
-mean:  [0.511 0.158 0.161 0.162 0.008] 

prediction gene Tcea1:
--max number of similar vector for min_distance 0.06: 8
- [[0.552 0.116 0.143 0.186 0.003]
 [0.521 0.174 0.152 0.127 0.026]
 [0.501 0.152 0.186 0.158 0.003]]
-mean:  [0.525 0.147 0.160 0.157 0.011] 

prediction gene Atp6v1h:
--max number of similar vector for min_distance 0.06: 9
- [[0.399 0.168 0.195 0.205 0.034]
 [0.424 0.236 0.151 0.175 0.014]]
-mean:  [0.411 0.202 0.173 0.190 0.024] 

prediction gene Rb1cc1:
--max number of similar vector for min_distance 0.06: 5
- [[0.337 0.160 0.171 0.269 0.063]
 [0.307 0.154 0.198 0.310 0.032]
 [0.369 0.164 0.170 0.245 0.051]
 [0.326 0.270 0.191 0

---min_distance: 0.055
--max number of similar vector for min_distance 0.055: 7
- [[0.834 0.018 0.060 0.087 0.001]
 [0.822 0.074 0.051 0.053 0.001]
 [0.827 0.045 0.056 0.058 0.014]]
-mean:  [0.828 0.046 0.055 0.066 0.005] 

prediction gene Als2:
--max number of similar vector for min_distance 0.06: 9
- [[0.113 0.207 0.295 0.375 0.010]
 [0.128 0.160 0.304 0.395 0.013]]
-mean:  [0.120 0.184 0.300 0.385 0.011] 

prediction gene Fzd7:
--max number of similar vector for min_distance 0.06: 10
- [[0.080 0.188 0.308 0.397 0.028]
 [0.046 0.284 0.291 0.370 0.009]
 [0.103 0.193 0.326 0.376 0.002]]
-mean:  [0.076 0.222 0.308 0.381 0.013] 

prediction gene Sumo1:
--max number of similar vector for min_distance 0.06: 11
- [[0.079 0.221 0.274 0.406 0.021]]
-mean:  [0.079 0.221 0.274 0.406 0.021] 

prediction gene Nop58:
--max number of similar vector for min_distance 0.06: 8
- [[0.101 0.246 0.263 0.348 0.042]
 [0.231 0.219 0.243 0.276 0.031]]
-mean:  [0.166 0.233 0.253 0.312 0.037] 

prediction gene 

--max number of similar vector for min_distance 0.06: 8
---min_distance: 0.055
--max number of similar vector for min_distance 0.055: 8
---min_distance: 0.05
--max number of similar vector for min_distance 0.05: 7
---min_distance: 0.045000000000000005
--max number of similar vector for min_distance 0.045000000000000005: 7
---min_distance: 0.04000000000000001
--max number of similar vector for min_distance 0.04000000000000001: 5
- [[0.023 0.157 0.344 0.465 0.011]]
-mean:  [0.023 0.157 0.344 0.465 0.011] 

prediction gene Retreg2:
--max number of similar vector for min_distance 0.06: 8
- [[0.107 0.241 0.279 0.364 0.009]
 [0.069 0.304 0.279 0.323 0.025]]
-mean:  [0.088 0.272 0.279 0.344 0.017] 

prediction gene Zfand2b:
--max number of similar vector for min_distance 0.06: 13
- [[0.181 0.175 0.300 0.311 0.035]
 [0.212 0.217 0.223 0.330 0.019]
 [0.111 0.168 0.305 0.345 0.071]
 [0.150 0.148 0.277 0.381 0.044]]
-mean:  [0.163 0.177 0.276 0.342 0.042] 

prediction gene Abcb6:
--max number of 

--max number of similar vector for min_distance 0.06: 11
- [[0.133 0.220 0.300 0.323 0.023]
 [0.139 0.192 0.287 0.380 0.002]
 [0.103 0.191 0.330 0.316 0.060]
 [0.114 0.188 0.325 0.333 0.040]]
-mean:  [0.122 0.198 0.311 0.338 0.031] 

prediction gene Ppip5k2:
--max number of similar vector for min_distance 0.06: 8
- [[0.079 0.162 0.300 0.458 0.001]]
-mean:  [0.079 0.162 0.300 0.458 0.001] 

prediction gene Gin1:
--max number of similar vector for min_distance 0.06: 5
---min_distance: 0.055
--max number of similar vector for min_distance 0.055: 5
- [[0.285 0.108 0.292 0.307 0.008]]
-mean:  [0.285 0.108 0.292 0.307 0.008] 

prediction gene Pam:
--max number of similar vector for min_distance 0.06: 10
- [[0.170 0.225 0.278 0.290 0.037]]
-mean:  [0.170 0.225 0.278 0.290 0.037] 

prediction gene B230216N24Rik:
--max number of similar vector for min_distance 0.06: 7
- [[0.504 0.102 0.196 0.195 0.004]]
-mean:  [0.504 0.102 0.196 0.195 0.004] 

prediction gene 1810006J02Rik:
--max number of sim

--max number of similar vector for min_distance 0.06: 4
---min_distance: 0.055
--max number of similar vector for min_distance 0.055: 4
- [[0.768 0.049 0.043 0.120 0.021]]
-mean:  [0.768 0.049 0.043 0.120 0.021] 

prediction gene Cd55:
--max number of similar vector for min_distance 0.06: 11
- [[0.220 0.152 0.258 0.335 0.036]
 [0.140 0.165 0.268 0.402 0.025]]
-mean:  [0.180 0.158 0.263 0.368 0.030] 

prediction gene Pfkfb2:
--max number of similar vector for min_distance 0.06: 9
- [[0.373 0.172 0.199 0.251 0.006]]
-mean:  [0.373 0.172 0.199 0.251 0.006] 

prediction gene Yod1:
--max number of similar vector for min_distance 0.06: 7
- [[0.135 0.180 0.295 0.351 0.039]
 [0.199 0.204 0.269 0.310 0.018]]
-mean:  [0.167 0.192 0.282 0.330 0.028] 

prediction gene Il10:
--max number of similar vector for min_distance 0.06: 13
- [[0.068 0.222 0.285 0.391 0.033]
 [0.140 0.125 0.298 0.405 0.032]
 [0.110 0.239 0.299 0.352 0.000]]
-mean:  [0.106 0.195 0.294 0.383 0.022] 

prediction gene Mapkapk2:


...

--max number of similar vector for min_distance 0.06: 10
- [[0.215 0.181 0.210 0.361 0.032]
 [0.207 0.180 0.284 0.320 0.009]]
-mean:  [0.211 0.180 0.247 0.341 0.021] 

prediction gene Terb1:
--max number of similar vector for min_distance 0.06: 6
---min_distance: 0.055
--max number of similar vector for min_distance 0.055: 6
- [[0.774 0.059 0.088 0.069 0.011]
 [0.761 0.047 0.101 0.085 0.006]]
-mean:  [0.768 0.053 0.094 0.077 0.009] 

prediction gene Nrn1l:
--max number of similar vector for min_distance 0.06: 7
- [[0.078 0.187 0.346 0.360 0.028]
 [0.096 0.180 0.289 0.385 0.051]]
-mean:  [0.087 0.183 0.317 0.373 0.039] 

prediction gene Esrp2:
--max number of similar vector for min_distance 0.06: 12
- [[0.087 0.172 0.272 0.423 0.045]]
-mean:  [0.087 0.172 0.272 0.423 0.045] 

prediction gene Slc38a8:
--max number of similar vector for min_distance 0.06: 11
- [[0.052 0.267 0.292 0.377 0.012]]
-mean:  [0.052 0.267 0.292 0.377 0.012] 

prediction gene Gm26812:
--max number of similar vecto

--max number of similar vector for min_distance 0.06: 7
- [[0.290 0.147 0.234 0.323 0.005]]
-mean:  [0.290 0.147 0.234 0.323 0.005] 

prediction gene Sectm1a:
--max number of similar vector for min_distance 0.06: 9
- [[0.127 0.252 0.261 0.331 0.028]]
-mean:  [0.127 0.252 0.261 0.331 0.028] 

prediction gene 9530020I12Rik:
--max number of similar vector for min_distance 0.06: 11
- [[0.128 0.192 0.270 0.371 0.039]
 [0.073 0.176 0.327 0.397 0.027]]
-mean:  [0.101 0.184 0.298 0.384 0.033] 

prediction gene Lratd1:
--max number of similar vector for min_distance 0.06: 7
- [[0.170 0.182 0.233 0.356 0.059]
 [0.152 0.193 0.311 0.275 0.069]
 [0.105 0.198 0.251 0.417 0.028]
 [0.160 0.139 0.304 0.395 0.002]
 [0.119 0.160 0.313 0.405 0.003]]
-mean:  [0.141 0.175 0.282 0.370 0.032] 

prediction gene 5730507C01Rik:
--max number of similar vector for min_distance 0.06: 11
- [[0.092 0.272 0.262 0.339 0.034]]
-mean:  [0.092 0.272 0.262 0.339 0.034] 

prediction gene Dcdc2c:
--max number of similar vect

--max number of similar vector for min_distance 0.06: 6
- [[0.374 0.178 0.208 0.222 0.019]]
-mean:  [0.374 0.178 0.208 0.222 0.019] 

prediction gene Gpd1:
--max number of similar vector for min_distance 0.06: 10
- [[0.182 0.147 0.273 0.359 0.039]]
-mean:  [0.182 0.147 0.273 0.359 0.039] 

prediction gene AU021092:
--max number of similar vector for min_distance 0.06: 12
- [[0.161 0.191 0.272 0.349 0.026]
 [0.129 0.219 0.285 0.365 0.001]
 [0.135 0.233 0.256 0.376 0.000]]
-mean:  [0.142 0.214 0.271 0.364 0.009] 

prediction gene Gm49584:
--max number of similar vector for min_distance 0.06: 9
- [[0.122 0.210 0.294 0.368 0.006]
 [0.162 0.143 0.291 0.376 0.028]
 [0.064 0.220 0.334 0.372 0.010]]
-mean:  [0.116 0.191 0.306 0.372 0.015] 

prediction gene Gm15651:
--max number of similar vector for min_distance 0.06: 10
- [[0.221 0.171 0.263 0.342 0.003]]
-mean:  [0.221 0.171 0.263 0.342 0.003] 

prediction gene Gm26562:
--max number of similar vector for min_distance 0.06: 7
- [[0.228 0.162 

--max number of similar vector for min_distance 0.06: 14
- [[0.041 0.197 0.331 0.412 0.019]
 [0.037 0.206 0.329 0.409 0.019]
 [0.011 0.228 0.358 0.403 0.001]]
-mean:  [0.030 0.210 0.339 0.408 0.013] 

prediction gene Gm29156:
--max number of similar vector for min_distance 0.06: 4
---min_distance: 0.055
--max number of similar vector for min_distance 0.055: 4
- [[0.760 0.085 0.073 0.056 0.027]
 [0.894 0.033 0.033 0.032 0.008]
 [0.859 0.064 0.035 0.028 0.013]]
-mean:  [0.838 0.061 0.047 0.039 0.016] 

prediction gene Ugt1a8:
--max number of similar vector for min_distance 0.06: 7
- [[0.520 0.107 0.147 0.188 0.038]
 [0.538 0.127 0.151 0.178 0.006]]
-mean:  [0.529 0.117 0.149 0.183 0.022] 

prediction gene Pla2g4a:
--max number of similar vector for min_distance 0.06: 11
- [[0.160 0.218 0.242 0.348 0.032]
 [0.153 0.165 0.269 0.375 0.038]
 [0.163 0.209 0.274 0.326 0.027]]
-mean:  [0.159 0.197 0.262 0.350 0.032] 

prediction gene Gm34342:
--max number of similar vector for min_distance 0.06

--max number of similar vector for min_distance 0.06: 12
- [[0.136 0.206 0.233 0.372 0.053]
 [0.131 0.234 0.267 0.328 0.041]
 [0.180 0.206 0.265 0.347 0.002]]
-mean:  [0.149 0.215 0.255 0.349 0.032] 

prediction gene Gjc2:
--max number of similar vector for min_distance 0.06: 11
- [[0.148 0.125 0.286 0.385 0.057]
 [0.206 0.142 0.262 0.345 0.045]
 [0.155 0.135 0.286 0.423 0.001]]
-mean:  [0.170 0.134 0.278 0.384 0.035] 

prediction gene Fbxw10:
--max number of similar vector for min_distance 0.06: 13
- [[0.062 0.159 0.321 0.435 0.023]
 [0.056 0.203 0.368 0.355 0.018]]
-mean:  [0.059 0.181 0.344 0.395 0.021] 

prediction gene Mgl2:
--max number of similar vector for min_distance 0.06: 10
- [[0.139 0.188 0.249 0.401 0.023]
 [0.138 0.157 0.307 0.395 0.002]]
-mean:  [0.139 0.173 0.278 0.398 0.013] 

prediction gene Nos2:
--max number of similar vector for min_distance 0.06: 11
- [[0.151 0.154 0.289 0.378 0.028]
 [0.103 0.171 0.310 0.409 0.007]]
-mean:  [0.127 0.163 0.300 0.393 0.017] 

pred

# save the output file

In [9]:
prediction_all_genes = np.array(prediction_all_genes)
prediction_all_genes[:,4] = 1- np.sum(prediction_all_genes[:,:4], axis=-1)

In [10]:
df = pd.DataFrame(prediction_all_genes, columns=['a_i','b_i','c_i','d_i','e_i'])
df['gene'] = g_name
df.index = df['gene']
df.drop(columns=['gene'], inplace=True)
df.to_csv('../part_c_output.csv')

In [11]:
ref = np.array([1,0,0,0,0])
l1_loss_progenitor = [np.sum(np.abs(v-ref)) for v in prediction_all_genes]
cycling_constraint = [0 if x < 0.05 else 1 for x in prediction_all_genes[:,3]]

df_a = pd.DataFrame({
    "gene": g_name,
    "objective": prediction_all_genes[:,0],
    "cycling_constraint": cycling_constraint,
    "l1_loss_progenitor": l1_loss_progenitor
})

df_a.sort_values(by='l1_loss_progenitor', ascending=True, inplace=True)

df_a.index = df_a['gene']
df_a.drop(columns=['gene','l1_loss_progenitor'], inplace=True)

df_a.to_csv('../../part_a/part_a_output.csv')

In [12]:
scaling_factor = np.array([1/0.0675, 1/0.2097, -1/0.3134, 1/0.3921, 0])
objective = [np.sum(v*scaling_factor) for v in prediction_all_genes]
cycling_constraint = [0 if x < 0.05 else 1 for x in prediction_all_genes[:,3]]

df_b = pd.DataFrame({
    "gene": g_name,
    "objective": objective,
    "cycling_constraint": cycling_constraint
})

df_b.sort_values(by='objective', ascending=False, inplace=True)

df_b.index = df_b['gene']
df_b.drop(columns=['gene'], inplace=True)

df_b.to_csv('../../part_b/part_b_output.csv')